# Lista de gastos



La lista que voy a crear a continuación es una forma manual de cargar datos escritos a mano. <br />
Además de cargar los datos y representarlos en un DataFrame de Pandas, voy a obtener representaciones visuales. <br />
Por último voy a desplegar esta notebook con el framework Mercury para disponibilizarlo en la web.

# 1) CARGAR Y EXPORTAR DATOS

In [1]:
CARGAR_DATOS = False

In [2]:
import os
import pandas as pd
import datetime as dt
import re

path = os.getcwd() + "\data\\"
os.makedirs(path, exist_ok=True)

# Informacion del momento de registro
now = dt.datetime.now()
year = now.year
date = lambda: now.strftime("%d/%m/%Y")
time = lambda: now.strftime("%H:%M")

if CARGAR_DATOS:
    variables = [
        "Fecha",
        "Descripcion",
        "Monto",
        "Categoria",
    ]
    N = len(variables)

    origin_row = list()
    for i in range(N):
        origin_row.append([])


    def cargar_datos():
        """Devuelve una lista de listas, cada lista esta una variable"""
        global origin_row, variables
        while True:
            i = iter(range(N))
            ii = iter(range(N))
            try:
                while True:
                    # Añadir Dato
                    origin_row[next(i)].append(input(f"{variables[next(ii)]}: "))
            except:
                pass
            if not input('Presione "Enter" para salir'):
                return origin_row


    dict_base = dict()
    columns_row = cargar_datos()
    for i, lista in enumerate(columns_row):
        if i == 0:
            for ii, elemento in enumerate(lista):
                lista[ii] = elemento + f"/{year}"
            dict_base[variables[i]] = lista
        dict_base[variables[i]] = lista

    # Descarga de datos en formato csv
    file_name = date().replace("/", "-")+'_'+time().replace(":", "-")
    pd.DataFrame(dict_base).to_csv(os.getcwd() + "\data\\" + file_name + '.csv')

# 2) LEER LOS DATOS ACONDICIONADOS Y PREPARARLOS

In [3]:
"""
Descripcion de las categorias:
    lifestyle: todos aquellos gastos que hacemos para darnos gustos personales
    frescos: principalmente carne y verdura, y otros productos de la canasta de frescos
    almacen: productos que pueden ser acopiados
    casa: incluye mascotas, farmacia, higiene personal y productos de limpieza
    gastos fijos: gastos que son corrientes, incluye nafta
    gastos extras: gastos que parecen fijos pero caducen en el tiempo
"""

file_names = os.listdir(path=os.getcwd() + "\data\\")


frame = pd.DataFrame()
for file in file_names:
    frame = pd.concat([frame, pd.read_csv(os.getcwd() + "\data\\" + file)])


frame.drop(columns=frame.columns[0], inplace=True)
frame.reset_index(drop=True, inplace=True)

patron="(error)"
regex = re.compile(patron)
match_id = []
for col_name in frame.columns:
    regex_search = frame[col_name].apply(lambda x: regex.search(str(x)))
    match_id.append(regex_search.dropna().apply(lambda x: x.group(0)).index)

all_matches = []
for variable in match_id:
    if len(variable) == 0:
        continue
    for match in variable:
        all_matches.append(match)

all_matches = list(set(all_matches))

frame.drop(all_matches, inplace=True)
frame.reset_index(drop=True, inplace=True)

frame;

 # 3) RESEARCH AND REPORT

In [4]:
from type_adjust import type_adjust
# https://towardsdev.com/how-to-use-plotly-as-pandas-plotting-backend-123ff5378003
pd.options.plotting.backend = "plotly"
data = type_adjust(frame)

index


In [9]:
data.groupby('Categoria')['Monto'].sum().plot(kind='bar', height=800)
